#Load Required Libraries 

In [ ]:
import zipfile
import glob
import os.path
from os import path
import cv2
import tensorflow
import keras
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
# from keras.applications.
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten, Dropout
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from tqdm import tqdm
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from re import search
import shutil

#Download and load dataset if not already present

In [ ]:
if not path.exists("agriculture-crop-images.zip") and not path.exists("dataset"):
  !gdown https://drive.google.com/uc?id=1WT3fiFl7ESqXkVr086zodbO8WRt7lvZk

if not path.exists("dataset"):
  with zipfile.ZipFile('agriculture-crop-images.zip', 'r') as zip_ref:
    zip_ref.extractall('dataset/')
  os.rename('dataset/some_more_images/some_more_images/Wheat', 'dataset/some_more_images/some_more_images/wheat')

  test_image_paths = (glob.glob("dataset/test_crop_image/*.*"))

  os.mkdir("dataset/validation_crop_images")

  classes = ["jute", "maize", "rice", "sugarcane", "wheat"]

  for image_path in test_image_paths:
    for sub_class in classes:
      if search(sub_class, image_path):
        test_path = "dataset/test_crop_image/" + sub_class
        if not path.exists(test_path):
          os.mkdir(test_path)
        shutil.copy2(image_path, test_path)
        os.remove(image_path)

  for sub_class in classes:
    images_paths = (glob.glob("dataset/kag2/"+sub_class+"/*.jgp"))
    for image_path in images_paths:
      shutil.copy2(image_path, "dataset/crop_images/"+sub_class)

  shutil.rmtree("dataset/kag2")

  paths = glob.glob("dataset/some_more_images/some_more_images/*")

  for directory_path in paths:
    shutil.move(directory_path, "dataset/validation_crop_images/")
  shutil.rmtree("dataset/some_more_images")  

Downloading...
From: https://drive.google.com/uc?id=1WT3fiFl7ESqXkVr086zodbO8WRt7lvZk
To: /content/agriculture-crop-images.zip
62.6MB [00:00, 72.3MB/s]


#Load VGG model, and load the previously trained weights

In [ ]:
model = VGG16(include_top=False, input_shape=(150, 150, 3), weights=None)
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(5, activation='softmax')(class1)

model = Model(inputs=model.inputs, outputs=output)
model.load_weights('pre_trained_weights/model_1_weights.h5')

## Display model summary

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0     

#Data Extractor

## Define the image generator for training validation and testing

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.0, zoom_range=0.2, horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

## Add paths for the datasets

In [ ]:
train_data_path = "dataset/crop_images"
validation_data_path = "dataset/validation_crop_images"
test_data_path = "dataset/test_crop_image"

##Using flow from directory, so the dataset is loaded in batches

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_data_path,
        target_size=(150, 150),
        batch_size=32,
        shuffle=True)

validation_generator = validation_datagen.flow_from_directory(
        validation_data_path,
        target_size=(150, 150),
        batch_size=32,
        shuffle=False)

test_generator = test_datagen.flow_from_directory(
        test_data_path,
        target_size=(150, 150),
        batch_size=32,
        shuffle=False)

Found 201 images belonging to 5 classes.
Found 50 images belonging to 5 classes.
Found 45 images belonging to 5 classes.


#Define the gradient descent rates

In [ ]:
from keras.optimizers import SGD
sgd = SGD(lr=0.0009, decay=1e-7, momentum=.8)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#Train VGG 16

In [ ]:
H = model.fit(train_generator, batch_size=64, epochs=15, validation_data=validation_generator)

Epoch 1/15
7/7 [==============================] - 20s 2s/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 7.4756 - val_accuracy: 0.3600
Epoch 2/15
7/7 [==============================] - 5s 657ms/step - loss: 7.1722e-04 - accuracy: 1.0000 - val_loss: 7.5292 - val_accuracy: 0.4000
Epoch 3/15
7/7 [==============================] - 5s 665ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 7.5812 - val_accuracy: 0.4000
Epoch 4/15
7/7 [==============================] - 5s 654ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 7.6517 - val_accuracy: 0.4000
Epoch 5/15
7/7 [==============================] - 4s 690ms/step - loss: 6.6005e-04 - accuracy: 1.0000 - val_loss: 7.8223 - val_accuracy: 0.4200
Epoch 6/15
7/7 [==============================] - 4s 683ms/step - loss: 6.9866e-04 - accuracy: 1.0000 - val_loss: 7.9038 - val_accuracy: 0.4200
Epoch 7/15
7/7 [==============================] - 4s 645ms/step - loss: 9.7439e-04 - accuracy: 1.0000 - val_loss: 7.9577 - val_accuracy: 0.4200
Epoch 

##Test accuracy

In [ ]:
score = model.evaluate(test_generator, batch_size=64)
print('Test Loss = ', score[0])
print('Test Accuracy = ', score[1])

2/2 [==============================] - 2s 1s/step - loss: 5.9112 - accuracy: 0.5333
Test Loss =  5.91122579574585
Test Accuracy =  0.5333333611488342


##Confusion matrix

In [ ]:
y_pred = model.predict(test_generator)
y_true = test_generator.classes
confusion_mtx = confusion_matrix(y_true, np.argmax(y_pred, axis=-1))
confusion_mtx

array([[ 4,  1,  1,  2,  1],
       [ 0,  0,  4,  2,  1],
       [ 1,  1,  5,  1,  0],
       [ 1,  1,  2,  4,  0],
       [ 0,  2,  0,  0, 11]])

#Define the ResNet Model and add load pre-trained weights

In [ ]:
model_2 = ResNet50(include_top=False, input_shape=(150, 150, 3), weights=None)
flat1 = Flatten()(model_2.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(5, activation='softmax')(class1)
model_2 = Model(inputs=model_2.inputs, outputs=output)
model_2.load_weights('pre_trained_weights/model_2_weights.h5')

##Show model summary

In [ ]:
model_2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 156, 156, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 75, 75, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 75, 75, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

##Compile the model

In [ ]:
model_2.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

##Train the model

In [ ]:
H2 = model_2.fit(train_generator, batch_size=64, epochs=15, validation_data=validation_generator)

Epoch 1/15
7/7 [==============================] - 15s 1s/step - loss: 0.0443 - accuracy: 0.9890 - val_loss: 5.2165 - val_accuracy: 0.4600
Epoch 2/15
7/7 [==============================] - 5s 661ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 5.1071 - val_accuracy: 0.4800
Epoch 3/15
7/7 [==============================] - 4s 697ms/step - loss: 2.3920e-04 - accuracy: 1.0000 - val_loss: 5.1165 - val_accuracy: 0.4400
Epoch 4/15
7/7 [==============================] - 4s 650ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 5.0417 - val_accuracy: 0.4400
Epoch 5/15
7/7 [==============================] - 4s 645ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 5.0537 - val_accuracy: 0.4600
Epoch 6/15
7/7 [==============================] - 4s 647ms/step - loss: 0.0146 - accuracy: 0.9971 - val_loss: 4.8429 - val_accuracy: 0.5000
Epoch 7/15
7/7 [==============================] - 5s 666ms/step - loss: 0.0281 - accuracy: 0.9929 - val_loss: 4.8034 - val_accuracy: 0.4600
Epoch 8/15
7/7 [==

##Test Accuracy

In [ ]:
score = model_2.evaluate(test_generator, batch_size=64)
print('Test Loss = ', score[0])
print('Test Accuracy = ', score[1])

2/2 [==============================] - 1s 675ms/step - loss: 4.4692 - accuracy: 0.5556
Test Loss =  4.469240188598633
Test Accuracy =  0.5555555820465088


##Confusion matrix

In [ ]:
y_pred = model_2.predict(test_generator)
y_true = test_generator.classes
confusion_mtx = confusion_matrix(y_true, np.argmax(y_pred, axis=-1))
confusion_mtx

array([[5, 1, 3, 0, 0],
       [0, 2, 4, 1, 0],
       [0, 0, 7, 1, 0],
       [0, 0, 5, 3, 0],
       [0, 0, 5, 0, 8]])

#Alex Net Model

In [ ]:
model_3 = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(150, 150, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5)
])

flat1 = Flatten()(model_3.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(5, activation='softmax')(class1)
model_3 = Model(inputs=model_3.inputs, outputs=output)
model_3.load_weights('pre_trained_weights/model_3_weights.h5')

##Show the summary of the model

In [ ]:
model_3.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5_input (InputLayer)  [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 35, 35, 96)        34944     
_________________________________________________________________
batch_normalization_5 (Batch (None, 35, 35, 96)        384       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 17, 17, 256)       614656    
_________________________________________________________________
batch_normalization_6 (Batch (None, 17, 17, 256)       1024      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 256)         0   

##Compile the model

In [ ]:
model_3.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

##Train the model

In [ ]:
H3 = model_3.fit(train_generator, batch_size=64, epochs=15, validation_data=validation_generator)

Epoch 1/15
7/7 [==============================] - 5s 614ms/step - loss: 0.0617 - accuracy: 0.9804 - val_loss: 4.2249 - val_accuracy: 0.5000
Epoch 2/15
7/7 [==============================] - 3s 441ms/step - loss: 0.0289 - accuracy: 0.9851 - val_loss: 4.6528 - val_accuracy: 0.4800
Epoch 3/15
7/7 [==============================] - 3s 443ms/step - loss: 0.0771 - accuracy: 0.9735 - val_loss: 4.6648 - val_accuracy: 0.4200
Epoch 4/15
7/7 [==============================] - 3s 425ms/step - loss: 0.0170 - accuracy: 0.9963 - val_loss: 5.2261 - val_accuracy: 0.4600
Epoch 5/15
7/7 [==============================] - 3s 428ms/step - loss: 0.0416 - accuracy: 0.9897 - val_loss: 5.0845 - val_accuracy: 0.4800
Epoch 6/15
7/7 [==============================] - 3s 436ms/step - loss: 0.0444 - accuracy: 0.9930 - val_loss: 4.9094 - val_accuracy: 0.4600
Epoch 7/15
7/7 [==============================] - 3s 463ms/step - loss: 0.0306 - accuracy: 0.9959 - val_loss: 4.9336 - val_accuracy: 0.4800
Epoch 8/15
7/7 [====

##Model accuracy

In [ ]:
score = model_3.evaluate(test_generator, batch_size=64)
print('Test Loss = ', score[0])
print('Test Accuracy = ', score[1])

2/2 [==============================] - 1s 396ms/step - loss: 3.2606 - accuracy: 0.5778
Test Loss =  3.260587453842163
Test Accuracy =  0.5777778029441833


##Confusion matrix

In [ ]:
y_pred = model_3.predict(test_generator)
y_true = test_generator.classes
confusion_mtx = confusion_matrix(y_true, np.argmax(y_pred, axis=-1))
confusion_mtx

array([[2, 1, 4, 1, 1],
       [0, 4, 1, 1, 1],
       [0, 0, 7, 1, 0],
       [0, 1, 2, 5, 0],
       [0, 4, 1, 0, 8]])